In [1]:
import numpy as np
# from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.pso import PSO
from scipy.interpolate import griddata
import pandas as pd


In [2]:
jpeg_cha = pd.read_csv("../../stable_tests/measurements/jpeg_snr_cha/characteristic.csv")

jpeg_cha = jpeg_cha[jpeg_cha["sparsity"]>0]
jpeg_cha = jpeg_cha[jpeg_cha["datasize_est"]>0]
tensor_size = 128*26*26 *4
jpeg_cha["ratio"] = tensor_size/jpeg_cha["datasize_est"]

In [3]:
pruning = [0.05, 0.1, 0.15, 0.2, 0.25,0.3,0.35]
quality = [60,70,80,90,100]

sample_points=[]
cmp_sample_values = []
snr_sample_values = []

cha_df_group =jpeg_cha.groupby("pruning_thresh")
for p in pruning:
    cha_df = cha_df_group.get_group(p)
    cha_quality_df = cha_df.groupby("quality")
    for q in quality:
        cha_plot_df= cha_quality_df.get_group(q)
        sample_points.append([q,p])
        cmp_sample_values.append(cha_plot_df["ratio"].mean())
        snr_sample_values.append(cha_plot_df["reconstruct_snr"].mean())

sample_points = np.array(sample_points)
cmp_sample_values = np.array(cmp_sample_values)
snr_sample_values = np.array(snr_sample_values)

In [14]:
# Interpolate the constraint function
def interpolated_constraint_cmp(xy):
    return griddata(sample_points, cmp_sample_values, (xy[0], xy[1]/100), method='cubic')

def interpolated_constraint_snr(xy):
    return griddata(sample_points, snr_sample_values, (xy[0], xy[1]/100), method='cubic')


class CustomProblem(ElementwiseProblem):

    def __init__(self,snr,cmp):
        self.snr = snr
        self.cmp = cmp
        super().__init__(n_var=2, n_obj=1, n_ieq_constr=2, xl=[60,5], xu=[100,35])

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = x[1]/35-x[0]/100
        out["G"] = np.array([
            self.cmp-interpolated_constraint_cmp(x),
            self.snr-interpolated_constraint_snr(x), 
        ])
        # penalty = 1000  # Large penalty factor
        # if constraint_1 > 0:  # If constraint is violated
        #     out["F"] += penalty * constraint_1

In [13]:
# Initialize the problem
problem = CustomProblem(snr=10,cmp=100)

# Select the algorithm (Genetic Algorithm in this case)
algorithm = PSO(pop_size=10)
# Perform the optimization
result = minimize(problem, algorithm, termination=('n_gen', 10),verbose=True)

# # Get and display the results
# optimal_x, optimal_y = result.X
# optimal_objective_value = -result.F[0]

print("Best solution found: \nX = %s\nF = %s" % (result.X, result.F))

n_gen  |  n_eval  |    f     |    S    |    w    |    c1    |    c2    |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |        - |       - |  0.9000 |  2.00000 |  2.00000 |  8.0214012364 |  3.437944E+01 |             - |             -
     2 |       20 | -6.7E-02 |       3 |  0.3588 |  2.00000 |  2.02048 |  6.3411491134 |  1.882893E+01 |             - |             -
     3 |       30 | -1.5E-02 |       3 |  0.3907 |  1.99006 |  2.03930 |  4.8844886203 |  1.207943E+01 |             - |             -
     4 |       40 |  0.06724 |       3 |  0.4426 |  1.97590 |  2.04546 |  4.8844886203 |  1.085521E+01 |             - |             -
     5 |       50 |  0.09793 |       3 |  0.4624 |  1.96586 |  2.05265 |  4.8844886203 |  6.1661491415 |             - |             -
     6 |       60 |  0.10336 |       3 |  0.4659 |  1.95753 |  2.06408 |  4.8711558551 |  5.9521775095 |             - |             -
     7 |       70 |  0.07299 |       3 |  0.4463 |  1.9

In [6]:
interpolated_constraint_cmp(result.X)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
interpolated_constraint_snr(result.X)

array(35.16830948)